In [1]:
from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from time import time
import d2lzh as d2l
import random
import sys 

# 04. 深度学习计算
上⼀章介绍了包括多层感知机在内的简单深度学习模型的原理和实现。本章我们将简要概括深度学习计算的各个重要组成部分，如模型构造、参数的访问和初始化、⾃定义层、以及读取、存储和使⽤GPU。

## 4.1 模型构造
让我们回顾⼀下在`多层感知机的简洁实现`⼀节中含单隐藏层的多层感知机的实现⽅法：
+ ⾸先构造`Sequential`实例
+ 依次添加两个全连接层，其中第⼀层的输出⼤小为256，即隐藏层单元个数是256；第⼆层的输出⼤小为10，即输出层单元个数是10

这⾥我们介绍另外⼀种基于Block类的模型构造⽅法：它让模型构造更加灵活。

### 4.1.1 继承Block类来构造模型
`Block`类是nn模块⾥提供的⼀个模型构造类，我们可以继承它来定义我们想要的模型。下⾯继承`Block`类构造本节开头提到的多层感知机。

In [2]:
class MLP(nn.Block):
    # 声明带有模型参数的层，这⾥声明了两个全连接层 
    def __init__(self, **kwargs):
        # 调⽤MLP⽗类Block的构造函数来进⾏必要的初始化。这样在构造实例时还可以指定其他函数 
        # 参数，如“模型参数的访问、初始化和共享”⼀节将介绍的模型参数params 
        super(MLP, self).__init__(**kwargs) 
        self.hidden = nn.Dense(256, activation='relu') # 隐藏层 
        self.output = nn.Dense(10) # 输出层
        
    # 定义模型的前向计算，即如何根据输⼊x计算返回所需要的模型输出 
    def forward(self, x):
        return self.output(self.hidden(x))

以上的MLP类中⽆须定义反向传播函数，系统将通过⾃动求梯度而⾃动⽣成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下⾯的代码初始化`net`并传⼊输⼊数据$X$做⼀次前向计算。其中，`net(X)`会调⽤MLP继承⾃`Block`类的`__call__`函数，这个函数将调⽤`MLP`类定义的`forward`函数来完成前向计算。

In [3]:
X = nd.random.uniform(shape=(2, 20)) 
net = MLP() 
net.initialize() 
net(X)


[[ 0.09543004  0.04614332 -0.00286654 -0.07790349 -0.05130243  0.02942037
   0.08696642 -0.0190793  -0.04122177  0.05088576]
 [ 0.0769287   0.03099705  0.00856576 -0.04467199 -0.06926839  0.09132434
   0.06786595 -0.06187842 -0.03436673  0.04234694]]
<NDArray 2x10 @cpu(0)>

> **注意**
> 这⾥并没有将Block类命名为Layer或者Model之类的名字，这是因为该类是⼀个可供⾃由组建的部件。它的⼦类既可以是⼀个层(如Gluon提供的Dense类)，⼜可以是⼀个模型(如这⾥定义的MLP类)，或者是模型的⼀个部分。我们下⾯通过两个例⼦来展⽰它的灵活性

### 4.1.2 Sequential类继承⾃Block类
`Block`类是一个通用的部件。事实上，`Sequential`类继承自`Block`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add`函数来逐一添加串联的`Block`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与Sequential类有相同功能的MySequential类。这或许可以帮助读者更加清晰地理解Sequential类的工作机制。

In [4]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        # block是⼀个Block⼦类实例，假设它有⼀个独⼀⽆⼆的名字。我们将它保存在Block类的 
        # 成员变量_children⾥，其类型是OrderedDict。当MySequential实例调⽤ 
        # initialize函数时，系统会⾃动对_children⾥所有成员初始化 
        self._children[block.name] = block

    def forward(self, x):
        # OrderedDict保证会按照成员添加时的顺序遍历成员 
        for block in self._children.values():
            x = block(x) 
        return x

我们⽤`MySequential`类来实现前⾯描述的MLP类，并使⽤随机初始化的模型做⼀次前向计算：

In [5]:
net = MySequential()
net.add(nn.Dense(256, activation='relu')) 
net.add(nn.Dense(10)) 
net.initialize() 
net(X)


[[ 0.00362228  0.00633332  0.03201144 -0.01369375  0.10336449 -0.03508018
  -0.00032164 -0.01676023  0.06978628  0.01303309]
 [ 0.03871715  0.02608213  0.03544959 -0.02521311  0.11005433 -0.0143066
  -0.03052466 -0.03852827  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

可以观察到这⾥`MySequential`类的使⽤跟`多层感知机的简洁实现`⼀节中`Sequential`类的使⽤没什么区别。

### 4.1.3 构造复杂的模型
虽然`Sequential`类可以使模型构造更加简单，且不需要定义`forward`函数，但直接继承`Block`类可以极⼤地拓展模型构造的灵活性。

下⾯我们构造⼀个稍微复杂点的⽹络`FancyMLP`。在这个⽹络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使⽤创建的常数参数外，我们还使⽤`NDArray`的函数和`Python`的控制流，并多次调⽤相同的层。

In [6]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs) 
        # 使⽤get_constant创建的随机权重参数不会在训练中被迭代(即常数参数)
        self.rand_weight = self.params.get_constant( 'rand_weight', nd.random.uniform(shape=(20, 20))) 
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, x):
        x = self.dense(x) 
        # 使⽤创建的常数参数，以及NDArray的relu函数和dot函数 
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1) 
        # 复⽤全连接层。等价于两个全连接层共享参数 
        x = self.dense(x) 
        # 控制流，这⾥我们需要调⽤asscalar函数来返回标量进⾏⽐较 
        while x.norm().asscalar() > 1:
            x /= 2 
        if x.norm().asscalar() < 0.8:
            x *= 10 
        return x.sum()

我们使⽤了常数权重`rand_weight`(注意它不是模型参数)、做了矩阵乘法操作并重复使⽤了相同的`Dense`层。下⾯我们来测试该模型的随机初始化和前向计算。

In [7]:
net = FancyMLP() 
net.initialize() 
net(X)


[18.571953]
<NDArray 1 @cpu(0)>

因为`FancyMLP`和`Sequential`类都是`Block`类的⼦类，所以我们可以嵌套调⽤它们。

In [8]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs) 
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'), nn.Dense(32, activation='relu')) 
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, x):
        return self.dense(self.net(x))

net = nn.Sequential() 
net.add(NestMLP(), nn.Dense(20), FancyMLP())
net.initialize() 
net(X)


[24.86621]
<NDArray 1 @cpu(0)>

## 4.2 模型参数的访问、初始化和共享
在`线性回归的简洁实现`⼀节中，我们通过`init`模块来初始化模型的全部参数。本节将深⼊讲解如何访问和初始化模型参数，以及如何在多个层之间共享同⼀份模型参数。

In [9]:
net = nn.Sequential() 
net.add(nn.Dense(256, activation='relu')) 
net.add(nn.Dense(10)) 
net.initialize() # 使⽤默认初始化⽅式

X = nd.random.uniform(shape=(2, 20))
Y = net(X) # 前向计算

### 4.2.1 访问模型参数
对于使⽤`Sequential`类构造的神经⽹络，我们可以通过⽅括号`[]`来访问⽹络的任⼀层。

对于`Sequential`实例中含模型参数的层，我们可以通过`Block`类的`params`属性来访问该层包含的所有参数。

In [10]:
net[0].params, type(net[0].params)

(dense10_ (
   Parameter dense10_weight (shape=(256, 20), dtype=float32)
   Parameter dense10_bias (shape=(256,), dtype=float32)
 ), mxnet.gluon.parameter.ParameterDict)

我们得到了⼀个由参数名称映射到参数实例的字典。其中权重参数的名称为`dense0_weight`，它由`net[0]`的名称(`dense0_`)和⾃⼰的变量名(`weight`)组成。参数的形状为`(256, 20)`，且数据类型为`float32`。

为了访问特定参数，我们既可以通过名字来访问字典⾥的元素，也可以直接使⽤它的变量名。下⾯两种⽅法是等价的，但通常后者的代码可读性更好。

In [11]:
net[0].params['dense10_weight'], net[0].weight

(Parameter dense10_weight (shape=(256, 20), dtype=float32),
 Parameter dense10_weight (shape=(256, 20), dtype=float32))

`Gluon`⾥参数类型为`Parameter`类，它包含参数和梯度的数值，可以分别通过`data`函数和`grad`函数来访问。因为我们随机初始化了权重，所以权重参数是⼀个由随机数组成的形状为`(256, 20)`的`NDArray`。

In [12]:
net[0].weight.data()


[[-0.06046963  0.00624272 -0.03472826 ... -0.01759475  0.0686483
  -0.06360765]
 [-0.01273243 -0.02659053 -0.04718638 ...  0.02570021  0.02275064
  -0.0166979 ]
 [-0.03555115  0.01875034  0.02322027 ...  0.06564643  0.04601197
  -0.01915742]
 ...
 [ 0.03173313  0.01789995  0.02519771 ... -0.06176154 -0.03986754
  -0.04898471]
 [ 0.00564718  0.04665586 -0.00028374 ...  0.05332779  0.02100175
  -0.06427249]
 [ 0.0438781   0.05357236  0.02753124 ...  0.04084889 -0.01963295
   0.05668835]]
<NDArray 256x20 @cpu(0)>

我们还没有进⾏反向传播计算，所以梯度的值全为0。

In [13]:
net[0].weight.grad()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 256x20 @cpu(0)>

我们可以访问其他层的参数，如输出层的偏差值。

In [14]:
net[1].bias.data()


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 10 @cpu(0)>

最后，可以使⽤`collect_params`函数来获取`net`变量所有嵌套的层所包含的所有参数。它返回的同样是⼀个由参数名称到参数实例的字典。

In [15]:
net.collect_params()

sequential2_ (
  Parameter dense10_weight (shape=(256, 20), dtype=float32)
  Parameter dense10_bias (shape=(256,), dtype=float32)
  Parameter dense11_weight (shape=(10, 256), dtype=float32)
  Parameter dense11_bias (shape=(10,), dtype=float32)
)

这个函数也可以通过正则表达式筛选需要的参数。

In [16]:
net.collect_params('.*weight')

sequential2_ (
  Parameter dense10_weight (shape=(256, 20), dtype=float32)
  Parameter dense11_weight (shape=(10, 256), dtype=float32)
)

### 4.2.2 初始化模型参数
模型的默认初始化⽅法是权重参数元素为`[0.07, 0.07]`之间均匀分布的随机数，偏差参数则全为0。

我们经常需要使⽤其他⽅法来初始化权重。`MXNet`的`init`模块⾥提供了多种预设的初始化⽅法。下⾯的例⼦中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [17]:
# ⾮⾸次对模型初始化需要指定force_reinit为真
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True) 
net[0].weight.data()[0]


[-0.00803235  0.01792648  0.00174623  0.01004736 -0.00177172  0.01704121
 -0.00315196 -0.0084464   0.00439474  0.00382651 -0.0071534  -0.01518173
 -0.00180067  0.01541854  0.00415876 -0.0093543   0.00476378 -0.00346679
  0.00468796  0.01868755]
<NDArray 20 @cpu(0)>

下⾯使⽤常数来初始化权重参数。

In [18]:
net.initialize(init=init.Constant(1), force_reinit=True) 
net[0].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 20 @cpu(0)>

如果只想对某个特定参数进⾏初始化，我们可以调⽤`Parameter`类的`initialize`函数， 它与`Block`类提供的`initialize`函数的使⽤⽅法⼀致。下例中我们对隐藏层的权重使⽤`Xavier`随机初始化⽅法。

In [19]:
net[0].weight.initialize(init=init.Xavier(), force_reinit=True) 
net[0].weight.data()[0]


[-0.10389185  0.07822403 -0.1289716  -0.1410463  -0.07610903 -0.10696874
 -0.01996909 -0.07058676  0.00648634  0.10942626  0.08052795 -0.09453681
  0.13527533 -0.01967503 -0.11284603 -0.05156991 -0.11588816  0.02459455
  0.02644953  0.12870744]
<NDArray 20 @cpu(0)>

### 4.2.3 ⾃定义初始化⽅法
可以通过实现`Initializer`类的⼦类自定义初始化方法。通常，我们只需要实现`_init_weight`这个函数，并将其传⼊的`NDArray`修改成初始化的结果。在下⾯的例⼦⾥，我们令权重有⼀半概率初始化为0，有另⼀半概率初始化为`[−10, −5]`和`[5, 10]`两个区间⾥均匀分布的随机数。

In [20]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        data[:] = nd.random.uniform(low=-10, high=10, shape=data.shape) 
        data *= data.abs() >= 5

net.initialize(MyInit(), force_reinit=True) 
net[0].weight.data()[0]

Init dense10_weight (256, 20)
Init dense11_weight (10, 256)



[ 7.142498  -9.206991  -7.6536884 -0.        -0.        -0.
 -0.        -5.294743  -0.        -8.822595   0.        -7.5231113
 -0.         7.135105   0.        -0.         0.        -0.
 -0.        -9.560527 ]
<NDArray 20 @cpu(0)>

我们还可以通过`Parameter`类的`set_data`函数来直接改写模型参数。例如，在下例中我们将隐藏层参数在现有的基础上加1。

In [21]:
net[0].weight.set_data(net[0].weight.data() + 1) 
net[0].weight.data()[0]


[ 8.142498  -8.206991  -6.6536884  1.         1.         1.
  1.        -4.294743   1.        -7.8225946  1.        -6.5231113
  1.         8.135105   1.         1.         1.         1.
  1.        -8.560527 ]
<NDArray 20 @cpu(0)>

### 4.2.4 共享模型参数
在有些情况下，我们希望在多个层之间共享模型参数。`模型构造`⼀节介绍了如何在`Block`类的`forward`函数⾥多次调⽤同⼀个层来计算。这⾥再介绍另外⼀种⽅法，它在构造层的时候指定使⽤特定的参数。如果不同层使⽤同⼀份参数，那么它们在前向计算和反向传播时都会共享相同的参数。

在下⾯的例⼦⾥，我们让模型的第⼆隐藏层(`shared变量`)和第三隐藏层共享模型参数。

In [22]:
net = nn.Sequential()
shared = nn.Dense(8, activation='relu') 
net.add(nn.Dense(8, activation='relu'), 
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10)) 
net.initialize()

X = nd.random.uniform(shape=(2, 20))
net(X)
net[1].weight.data()[0] == net[2].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 8 @cpu(0)>

我们在构造第三隐藏层时通过`params`来指定它使⽤第⼆隐藏层的参数。因为模型参数⾥包含了梯度，所以在反向传播计算时，第⼆隐藏层和第三隐藏层的梯度都会被累加在`shared.params.grad()`⾥。


## 4.3 模型参数的延后初始化
如果做了上⼀节练习，你会发现模型`net`在调⽤初始化函数`initialize`之后、在做前向计算`net(X)`之前时，权重参数的形状中出现了0。虽然直觉上`initialize`完成了所有参数初始化过程，然而这在`Gluon`中却是不⼀定的。

### 4.3.1 延后初始化
也许读者早就注意到了，在之前使⽤`Gluon`创建的全连接层都没有指定输⼊个数。例如，在上⼀节使⽤的多层感知机`net`⾥，我们创建的隐藏层仅仅指定了输出⼤小为256。当调⽤`initialize`函数时，由于隐藏层输⼊个数依然未知，系统也⽆法得知该层权重参数的形状。只有在当我们将形状是`(2, 20)`的输⼊`X`传进⽹络做前向计算`net(X)`时，系统才推断出该层的权重参数形状为`(256, 20)`。因此，这时候我们才能真正开始初始化参数。

In [23]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape) # 实际的初始化逻辑在此省略了

net = nn.Sequential() 
net.add(nn.Dense(256, activation='relu'), nn.Dense(10))
net.initialize(init=MyInit())

注意，`initialize`函数执⾏完并未打印任何信息。由此可⻅，调⽤`initialize`函数时并没有真正初始化参数。下⾯我们定义输⼊并执⾏⼀次前向计算。

In [24]:
X = nd.random.uniform(shape=(2, 20)) 
Y = net(X)

Init dense16_weight (256, 20)
Init dense17_weight (10, 256)


在根据输⼊`X`做前向计算时，系统能够根据输⼊的形状⾃动推断出所有层的权重参数的形状。系统在创建这些参数之后，调⽤`MyInit`实例对它们进 ⾏初始化，然后才进⾏前向计算。

当然，这个初始化只会在第⼀次前向计算时被调⽤。之后我们再运⾏前向计算`net(X)`时则不会重新初始化。

系统将真正的参数初始化延后到获得⾜够信息时才执⾏的⾏为叫作`延后初始化`。它可以让模型的创建更加简单：只需要定义每个层的输出⼤小，而不⽤⼈⼯推测它们的输⼊个数。这对于之后将介绍的定义多达数⼗甚⾄数百层的⽹络来说尤其⽅便。

然而，任何事物都有两⾯性。正如本节开头提到的那样，延后初始化也可能会带来⼀定的困惑。 在第⼀次前向计算之前，我们⽆法直接操作模型参数，例如⽆法使⽤`data`函数和`set_data`函数来获取和修改参数。因此，我们经常会额外做⼀次前向计算来迫使参数被真正地初始化。

### 4.3.2 避免延后初始化
如果系统在调⽤`initialize`函数时能够知道所有参数的形状，那么延后初始化就不会发⽣。

第⼀种情况是我们要对已初始化的模型重新初始化时。因为参数形状不会发⽣变化，所以系统能够⽴即进⾏重新初始化。

In [25]:
net.initialize(init=MyInit(), force_reinit=True)

Init dense16_weight (256, 20)
Init dense17_weight (10, 256)


第⼆种情况是我们在创建层的时候指定了它的输⼊个数，使系统不需要额外的信息来推测参数形状。下例中我们通过`in_units`来指定每个全连接层的输⼊个数，使初始化能够在`initialize`函数被调⽤时⽴即发⽣。

In [26]:
net = nn.Sequential()
net.add(nn.Dense(256, in_units=20, activation='relu')) 
net.add(nn.Dense(10, in_units=256))
net.initialize(init=MyInit())

Init dense18_weight (256, 20)
Init dense19_weight (10, 256)


## 4.4 ⾃定义层
深度学习的⼀个魅⼒在于神经⽹络中各式各样的层，例如全连接层和后⾯章节中将要介绍的卷积层、池化层与循环层。虽然Gluon提供了⼤量常⽤的层，但有时候我们依然希望⾃定义层。

### 4.4.1 不含模型参数的⾃定义层
我们先介绍如何定义⼀个不含模型参数的⾃定义层。事实上，这和`模型构造`⼀节中介绍的使⽤`Block`类构造模型类似。下⾯的`CenteredLayer`类通过继承`Block`类⾃定义了⼀个将输⼊减掉均值后输出的层，并将层的计算定义在了`forward`函数⾥。这个层⾥不含模型参数。

In [27]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)

    def forward(self, x):
        return x - x.mean()

layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

我们也可以⽤它来构造更复杂的模型：

In [28]:
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())

下⾯打印⾃定义层各个输出的均值。因为均值是浮点数，所以它的值是⼀个很接近0的数。

In [29]:
net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

-3.6379788e-11

### 4.4.2 含模型参数的⾃定义层
我们还可以⾃定义含模型参数的⾃定义层。其中的模型参数可以通过训练学出。

在⾃定义含模型参数的层时，我们可以利⽤`Block`类⾃带的`ParameterDict`类型的成员变量`params`。 它是⼀个由字符串类型的参数名字映射到`Parameter`类型的模型参数的字典。我们可以通过`get`函数从`ParameterDict`创建`Parameter`实例。

In [30]:
params = gluon.ParameterDict()
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

现在我们尝试实现⼀个含权重参数和偏差参数的全连接层。它使⽤ReLU函数作为激活函数。

In [31]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输⼊个数 
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs) 
        self.weight = self.params.get('weight', shape=(in_units, units)) 
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data() 
        return nd.relu(linear)

实例化MyDense类并访问它的模型参数。

In [32]:
dense = MyDense(units=3, in_units=5) 
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

直接使⽤⾃定义层做前向计算。

In [33]:
dense.initialize() 
dense(nd.random.uniform(shape=(2, 5)))


[[0.04578072 0.05299256 0.02392543]
 [0.05540341 0.05370332 0.01740249]]
<NDArray 2x3 @cpu(0)>

我们也可以使⽤⾃定义层构造模型。它和`Gluon`的其他层在使⽤上很类似。

In [34]:
net = nn.Sequential()
net.add(MyDense(8, in_units=64), MyDense(1, in_units=8)) 
net.initialize() 
net(nd.random.uniform(shape=(2, 64)))


[[0.04486297]
 [0.05108206]]
<NDArray 2x1 @cpu(0)>

## 4.5 读取和存储
我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使⽤。

### 4.5.1 读写NDArray
使⽤`save`函数和`load`函数分别存储和读取NDArray。 

```python
x = nd.ones(3) 
nd.save('x', x)

x2 = nd.load('x')
```

可以存储⼀列NDArray并读回内存。

```python
y = nd.zeros(4) 
nd.save('xy', [x, y])

x2, y2 = nd.load('xy')
```

我们甚⾄可以存储并读取⼀个从字符串映射到NDArray的字典。
```python
mydict = {'x': x, 'y': y}
nd.save('mydict', mydict) 
mydict2 = nd.load('mydict')
```

### 4.5.2 读写Gluon模型的参数
除NDArray以外， 我们还可以读写`Gluon`模型的参数。`Block`类提供了`save_parameters`函数和`load_parameters`函数来读写模型参数。我们先创建⼀个多层感知机，并将其初始化。由于延后初始化，我们需要先运⾏⼀次前向计算才能实际初始化模型参数。

In [35]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu') 
        self.output = nn.Dense(10)
        
    def forward(self, x):
        return self.output(self.hidden(x))
    
net = MLP() 
net.initialize()
X = nd.random.uniform(shape=(2, 20))
Y = net(X)

下⾯把该模型的参数存成⽂件：

```python
filename = 'mlp.params' 
net.save_parameters(filename)
```

再实例化⼀次定义好的多层感知机。与随机初始化模型参数不同，我们在这⾥直接读取保存在⽂件⾥的参数。

```python
net2 = MLP()
net2.load_parameters(filename)
```

因为这两个实例都有同样的模型参数，那么对同⼀个输⼊$X$的计算结果将会是⼀样的。我们来验证⼀下。

```python
Y2 = net2(X) 
Y2 == Y
```

## 4.6 GPU计算
[todo]